In [1]:
import random
import torch
import numpy as np
import pandas as pd

In [36]:
from models.neural_networks.utils import prepare_data_for_predict
from features.build_features import calculate_expenses
from data.utils import load_default_data
from features.build_features import aggregate_users_data
from models.neural_networks.NeuralNetworkRegressor import NeuralNetworkRegressor
from models.neural_networks.SimpleRegressor import SimpleRegressor
from models.neural_networks.utils import train

In [37]:
seed = 213769420
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [38]:
train_sessions, test, products, users, deliveries = load_default_data(prefix="../../data/")
targets = calculate_expenses(test, products, users)
targets

,user_id,expenses
0,102,0.00
1,103,554.77
2,104,2332.01
3,105,0.00
4,106,0.00
...,...,...
195,297,109.00
196,298,2399.00
197,299,0.00
198,300,0.00


In [39]:
targets['expenses'].mean()

656.25255

In [40]:
# targets['expenses'] = targets['expenses'].apply(lambda b: 1 if b > 5000 else 0) binarization
targets['expenses'] = targets['expenses'].apply(lambda x: x)

In [41]:
some_net = SimpleRegressor()
users_data = aggregate_users_data(train_sessions, users, products)

In [42]:
train(some_net, users_data, targets)

Epoch 0 loss 2.74e+06
Epoch 1 loss 2.83e+06
Epoch 2 loss 3.39e+06
Epoch 3 loss 2.35e+06
Epoch 4 loss 1.93e+06
Epoch 5 loss 2.33e+06
Epoch 6 loss 2.23e+06
Epoch 7 loss 2.11e+06
Epoch 8 loss 2.66e+06
Epoch 9 loss 1.78e+06
Epoch 10 loss 1.74e+06
Epoch 11 loss 2.11e+06
Epoch 12 loss 1.67e+06
Epoch 13 loss 1.75e+06
Epoch 14 loss 1.62e+06
Epoch 15 loss 1.46e+06
Epoch 16 loss 2.18e+06
Epoch 17 loss 2.34e+06
Epoch 18 loss 2.02e+06
Epoch 19 loss 1.55e+06
Epoch 20 loss 2.31e+06
Epoch 21 loss 1.49e+06
Epoch 22 loss 2.11e+06
Epoch 23 loss 1.44e+06
Epoch 24 loss 1.85e+06
Epoch 25 loss 1.54e+06
Epoch 26 loss 1.51e+06
Epoch 27 loss 1.62e+06
Epoch 28 loss 1.49e+06
Epoch 29 loss 1.84e+06
Epoch 30 loss 1.65e+06
Epoch 31 loss 1.9e+06
Epoch 32 loss 1.57e+06
Epoch 33 loss 1.82e+06
Epoch 34 loss 2.03e+06
Epoch 35 loss 1.57e+06
Epoch 36 loss 1.62e+06
Epoch 37 loss 2.47e+06
Epoch 38 loss 2.29e+06
Epoch 39 loss 1.51e+06
Epoch 40 loss 1.44e+06
Epoch 41 loss 1.63e+06
Epoch 42 loss 2.02e+06
Epoch 43 loss 1.56e+06

In [43]:
extracted_users_data = aggregate_users_data(train_sessions, users, products)
x, cat_x = prepare_data_for_predict(extracted_users_data)
x = torch.from_numpy(x.values).float()
cat_x = torch.from_numpy(cat_x.values).float()
some_net.eval()
out = some_net(x, cat_x).squeeze()
out = out.detach().numpy()

In [44]:
# out_array = {int(user_id): out[i] > 0.3 for i, user_id in enumerate(extracted_users_data["user_id"].to_list())} binarization
out_array = {int(user_id): out[i] for i, user_id in enumerate(extracted_users_data["user_id"].to_list())}

In [45]:
evaluation_array = {int(user_id): targets['expenses'].mean() for i, user_id in enumerate(extracted_users_data["user_id"].to_list())}

In [46]:
def loss(targets, out_array):
    sum = 0
    for row in targets.values[:]:
        sum+= (row[1] - out_array[row[0]])**2

    return sum

In [47]:
loss(targets, out_array)

348234629.8035757

In [48]:
loss(targets, evaluation_array)

354843797.56659997

In [49]:
torch.save(some_net.state_dict(), "../parameters/simple_v1")